### - Sobhan Moradian Daghigh
#### - 4/24/2022
#### - ANN - HW03

In [32]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

### Q3.

In [38]:
x_train = pd.read_csv('./Dataset/train/X_train.txt', header=None, delim_whitespace=True)
y_train = pd.read_csv('./Dataset/train/y_train.txt', header=None, delim_whitespace=True)

x_dataset = pd.read_csv('./Dataset/test/X_test.txt', header=None, delim_whitespace=True)
y_dataset = pd.read_csv('./Dataset/test/y_test.txt', header=None, delim_whitespace=True)

x_test, x_val, y_test, y_val = train_test_split(x_dataset, y_dataset, test_size=0.3, random_state=1)

In [39]:
(x_train.shape, x_test.shape, x_val.shape)

((7352, 561), (2062, 561), (885, 561))

### Q4.